In [1]:
import numpy as np
import pandas as pd
from sklearn.svm import SVC
from imblearn.over_sampling import SMOTE
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score, roc_curve
from collections import Counter
import matplotlib.pyplot as plt


# Carregar os arquivos
train_df = pd.read_csv('C:\\Users\\Carlos Moura\\Desktop\\Modelos\\train.csv')
test_df = pd.read_csv('C:\\Users\\Carlos Moura\\Desktop\\Modelos\\test.csv')

# Definir o limite de 50% para exclusão de colunas
threshold = 0.5

# Remover colunas com mais de 50% de valores ausentes em ambos os datasets
train_df = train_df.loc[:, train_df.isnull().mean() <= threshold]

# Alinhar as colunas entre treino e teste (manter apenas colunas comuns)
common_columns = train_df.columns.intersection(test_df.columns)
train_df = train_df[common_columns]
test_df = test_df[common_columns]

# Calcular a mediana das colunas do conjunto de treino
train_medians = train_df.median()

# Preencher os valores ausentes no conjunto de treino usando a mediana calculada
train_df.fillna(train_medians, inplace=True)

# Preencher os valores ausentes no conjunto de teste usando a mediana do conjunto de treino
test_df.fillna(train_medians, inplace=True)

# Função para substituir outliers pela mediana
def replace_outliers_with_median(df, lower_quantile=0.25, upper_quantile=0.75, factor=1.5):
    for col in df.select_dtypes(include=np.number).columns:
        Q1 = df[col].quantile(lower_quantile)
        Q3 = df[col].quantile(upper_quantile)
        IQR = Q3 - Q1
        lower_bound = Q1 - factor * IQR
        upper_bound = Q3 + factor * IQR
        median = df[col].median()
        
        # Substituir os valores que estão fora dos limites pelo valor da mediana
        df[col] = np.where((df[col] < lower_bound) | (df[col] > upper_bound), median, df[col])
    return df

# Aplicar a função de substituição dos outliers nos dados de treino e teste
X_train = replace_outliers_with_median(train_df.drop('y', axis=1))
y_train = train_df['y']
X_test = replace_outliers_with_median(test_df.drop('y', axis=1))
y_test = test_df['y']

# Escalonar os dados usando StandardScaler
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Converter os arrays escalonados de volta para DataFrames
X_train_scaled = pd.DataFrame(X_train_scaled, columns=X_train.columns)
X_test_scaled = pd.DataFrame(X_test_scaled, columns=X_test.columns)

In [ ]:
# Treinar o SVM com os parâmetros encontrados (C=10, gamma='scale')
svm_rbf = SVC(kernel='rbf', C=10, gamma='scale', class_weight='balanced', random_state=42, probability=True)
svm_rbf.fit(X_train_scaled, y_train)

# Fazer previsões no conjunto de treino balanceado
y_pred_train = svm_rbf.predict(X_train_scaled)
y_pred_train_prob = svm_rbf.predict_proba(X_train_scaled)[:, 1]

# Avaliar o desempenho do modelo no conjunto de treino
print("\nRelatório de classificação no conjunto de treino:")
print(classification_report(y_train, y_pred_train))

# Matriz de confusão no conjunto de treino
print("\nMatriz de confusão no conjunto de treino:")
print(confusion_matrix(y_train, y_pred_train))

# Calcular a AUC-ROC no conjunto de treino
auc_train = roc_auc_score(y_train, y_pred_train_prob)
print(f"AUC-ROC no conjunto de treino: {auc_train}")

# Curva ROC no conjunto de treino
fpr_train, tpr_train, _ = roc_curve(y_train, y_pred_train_prob)
plt.figure()
plt.plot(fpr_train, tpr_train, label=f'AUC-ROC (treino) = {auc_train:.2f}')
plt.xlabel('Taxa de Falsos Positivos (FPR)')
plt.ylabel('Taxa de Verdadeiros Positivos (TPR)')
plt.title('Curva ROC - Treino')
plt.legend(loc='lower right')
plt.show()


In [ ]:
# Fazer previsões no conjunto de teste usando o modelo treinado
y_pred_test = svm_rbf.predict(X_test_scaled)
y_pred_test_prob = svm_rbf.predict_proba(X_test_scaled)[:, 1]

# Avaliar o desempenho do modelo no conjunto de teste
print("\nRelatório de classificação no conjunto de teste:")
print(classification_report(y_test, y_pred_test))

# Matriz de confusão no conjunto de teste
print("\nMatriz de confusão no conjunto de teste:")
print(confusion_matrix(y_test, y_pred_test))

# Calcular a AUC-ROC no conjunto de teste
auc_test = roc_auc_score(y_test, y_pred_test_prob)
print(f"AUC-ROC no conjunto de teste: {auc_test}")

# Curva ROC no conjunto de teste
fpr_test, tpr_test, _ = roc_curve(y_test, y_pred_test_prob)
plt.figure()
plt.plot(fpr_test, tpr_test, label=f'AUC-ROC (teste) = {auc_test:.2f}')
plt.xlabel('Taxa de Falsos Positivos (FPR)')
plt.ylabel('Taxa de Verdadeiros Positivos (TPR)')
plt.title('Curva ROC - Teste')
plt.legend(loc='lower right')
plt.show()
